## Iggy demo with kepler.gl visualization

This notebook gives a quick example of how to enrich some user data with Iggy and then visualize the resulting features using [kepler.gl](https://kepler.gl/).

We're assuming you have some Iggy data on hand (if not, you can download a sample [here](https://docs.askiggy.com/download/sample-data)) and have unzipped the downloaded package into a local directory like so:

```bash
tar xzvf iggy-package-wkt-20211110214810_fl_pinellas_quadkeys.tar.gz
```

The resulting data (parquet files) should then be accessible in the directory `iggy-package-wkt-20211110214810_fl_pinellas_quadkeys`. 

Ok? Let's go!

In [ ]:
# Install dependencies if needed
!pip install pandas geopandas pyarrow shapely keplergl

: 

In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point, Polygon
from keplergl import KeplerGl

### Let's start with a dataset

We'll assume you're using Iggy to enrich some data you already have on hand, like a data set of properties your company holds or number of users by zip code.

For this demo, we'll start with a 2018 dataset of public pools in FL which can be downloaded [here](https://download.fgdl.org/pub/state/public_pools_mar18.zip). The next two code blocks download the data, read it, and transform it into a data frame where each row represents a zip code and a column indicates the number of public pools in that zip.

In [3]:
!wget https://download.fgdl.org/pub/state/public_pools_mar18.zip
!unzip public_pools_mar18.zip

--2022-03-03 03:48:09--  https://download.fgdl.org/pub/state/public_pools_mar18.zip
Resolving download.fgdl.org (download.fgdl.org)... 128.227.148.144
Connecting to download.fgdl.org (download.fgdl.org)|128.227.148.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4409038 (4.2M) [application/zip]
Saving to: ‘public_pools_mar18.zip.2’

public_pools_mar18. 100%[===================>]   4.20M   664KB/s    in 18s     

2022-03-03 03:48:33 (242 KB/s) - ‘public_pools_mar18.zip.2’ saved [4409038/4409038]

Archive:  public_pools_mar18.zip
replace public_pools_mar18.dbf? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [2]:
# read data
pools_gdf = gpd.read_file('public_pools_mar18.shp')
# calculate pool counts by zip
pools = pd.DataFrame(pools_gdf)[["ZIP_CODE", "ENTITYNUMB"]].groupby(["ZIP_CODE"]).count()
pools.rename(columns={"ENTITYNUMB": "pool_count"}, inplace=True)

So we're starting with a simple data frame that has two columns: `ZIP_CODE` (which is currently the index), and `pool_count`. If you're following along with your own data, just get it to the point where you have it in a pandas DataFrame and go from here.

In [3]:
pools.head()

,pool_count
ZIP_CODE,
32003,26
32008,2
32024,4
32025,10
32033,5


### Loading and enriching with Iggy data

The next step is to load the Iggy data that we'll use to enrich our pools dataset.

In [4]:
VERSION_ID = "20211110214810"
PREFIX = "fl_pinellas_quadkeys"

In [5]:
print(pools.head())

          pool_count
ZIP_CODE            
32003             26
32008              2
32024              4
32025             10
32033              5


In [6]:
# Load Iggy zipcode data and set the dataframe index to the `id` which contains the zip code
iggy_zips = pd.read_parquet(f"../iggy-data/iggy-package-wkt-{VERSION_ID}_{PREFIX}/{PREFIX}_zipcode_{VERSION_ID}")
iggy_zips.set_index("id", inplace=True)
print(pools)

# Merge it with our pools
# iggy_pools = pools.merge(iggy_zips, left_index=True, right_index=True, suffixes=("", "_zipcode"))
# print(iggy_pools.columns.tolist())
iggy_pools = pools.join(iggy_zips, how="inner")
print(iggy_pools)

          pool_count
ZIP_CODE            
32003             26
32008              2
32024              4
32025             10
32033              5
...              ...
34991              1
34994             74
34995              1
34996             78
34997             67

[897 rows x 1 columns]
       pool_count   name   area_sqkm  perimeter_km  population  poi_count  \
33556          12  33556  106.265117     58.167026       23182        140   
33626          36  33626   38.051243     34.312747       30743        306   
33635          26  33635   16.020221     30.481792       18650        134   
33701          66  33701   10.382468     16.786176       15728        788   
33702          65  33702   33.046359     35.880793       32019        437   
33703          27  33703   17.992311     22.703142       25699        287   
33704          11  33704   11.719535     19.348042       16982        313   
33705          22  33705   21.885304     29.875739       28857        548   
33706     

Great! Now we have a dataframe with not one but 227 columns that describe each zip code.

If you noticed, the number of rows in this dataset shrunk from 897 to 52. This is because our Iggy sample dataset only contains 52 zip codes in Pinellas County. 

### Visualization

Next let's visualize the enriched pools data using kepler.

The first thing we'll need to do is turn our vanilla DataFrame into a GeoDataFrame, so that kepler can understand the geometries.

In [8]:
# 'pool_count' in iggy_pools.columns
# iggy_pools.filter(like='pool_count')
# print(iggy_pools.columns.tolist())
# print(iggy_pools.index)

In [9]:
# Define the geometry
pools_geom = iggy_pools.pop("geometry")
pools_geom = gpd.GeoSeries(pools_geom.map(lambda geom: wkt.loads(geom)), crs="WGS84")

# Convert to GeoDataFrame
iggy_pools_gdf = gpd.GeoDataFrame(iggy_pools, geometry=pools_geom)
iggy_pools_gdf.fillna(0, inplace=True)

/content/env/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)


Finally, let's take a look!

In [10]:
map = KeplerGl()
map.add_data(data=iggy_pools_gdf, name="enriched_pools")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [13]:
map

KeplerGl(data={'enriched_pools': {'index': ['33556', '33626', '33635', '33701', '33702', '33703', '33704', '33…

In [12]:
map.save_to_html(file_name='first_map.html')

Map saved to first_map.html!


When you first load the map, it'll show you the outlines of zip codes in Pinellas County and every zip will have the same color.

You can change the colors to reflect features by:
- clicking on the little arrow in the top left of the map
- clicking the down arrow at the right side of the `enriched_pools` box 
- clicking on the three dots by "Fill Color"
- selecting a feature in the "Color Based On" box 

### Exporting

Now let's say you prefer to use kepler.gl in their web app, instead of here in the notebook. Or, maybe you haven't been able to get the kepler map to render in this notebook and have exhausted all of the install suggestions in [kepler's docs](https://docs.kepler.gl/docs/keplergl-jupyter) and just want to see the data. 

You can export your GeoDataFrame to a file in GeoJSON format, which you can then drag and drop into your kepler browser window:

In [60]:
print(iggy_pools_gdf)
# iggy_pools_gdf.to_file("iggy-pools-export.json", driver="GeoJSON", index=True)

       pool_count   name   area_sqkm  perimeter_km  population  poi_count  \
33556          12  33556  106.265117     58.167026       23182        140   
33626          36  33626   38.051243     34.312747       30743        306   
33635          26  33635   16.020221     30.481792       18650        134   
33701          66  33701   10.382468     16.786176       15728        788   
33702          65  33702   33.046359     35.880793       32019        437   
33703          27  33703   17.992311     22.703142       25699        287   
33704          11  33704   11.719535     19.348042       16982        313   
33705          22  33705   21.885304     29.875739       28857        548   
33706         274  33706   24.828421     32.731203       16321        434   
33707          69  33707   19.237158     21.020447       24767        522   
33708         153  33708   20.599742     34.535818       16083        317   
33709          36  33709   14.553476     23.611285       24619        454   